ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже
Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. 
Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов),
а не отложенном датасете.

In [2]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [5]:
 ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# группируем таблицы
# (для метода join сначала создадим, а затем удалим индекс по полю movieId, заодно удалив пропуски и переписав датасет)
movies_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)
movies_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [12]:
# Выберем наугад пользователя с его рейтингами 
movies_ratings[movies_ratings.userId == 100].title.unique()

array(['Grumpier Old Men (1995)', 'American President, The (1995)',
       'Casino (1995)', 'Sense and Sensibility (1995)',
       'Ace Ventura: When Nature Calls (1995)', 'Persuasion (1995)',
       "Mr. Holland's Opus (1995)", 'Bed of Roses (1996)',
       'Nick of Time (1995)', 'Broken Arrow (1996)',
       'Happy Gilmore (1996)', 'First Knight (1995)',
       'Circle of Friends (1995)', 'Clerks (1994)', 'Exit to Eden (1994)',
       'Ed Wood (1994)', 'Forget Paris (1995)', 'Little Women (1994)',
       'Legends of the Fall (1994)', 'Pulp Fiction (1994)',
       'Forrest Gump (1994)', 'Four Weddings and a Funeral (1994)',
       'Maverick (1994)', 'Speed (1994)', 'True Lies (1994)',
       'Dazed and Confused (1993)', 'Much Ado About Nothing (1993)',
       'Philadelphia (1993)', 'Sirens (1994)',
       'Sleepless in Seattle (1993)', 'Tombstone (1993)',
       'True Romance (1993)', 'Ghost (1990)', 'Dances with Wolves (1990)',
       'Batman (1989)', 'Snow White and the Seven Dwarfs

In [14]:
# Создадим датафрейм Pandas 
dataset = pd.DataFrame({
    'uid': movies_ratings.userId,       # id пользователя
    'iid': movies_ratings.title,        # название фильма
    'rating': movies_ratings.rating     # рейтинг, который пользователь поставил фильму 
})
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [15]:
# находим разброс значений по рейтингам в датафрейме по рейтингу
print(ratings.rating.min())
print(ratings.rating.max())

0.5
5.0


In [16]:
# Объявим переменную класса Reader и преобразуем датафрейм Pandas в датасет surprise 
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [17]:
# Разобьем датасет surprise на трейн и тест 
trainset, testset = train_test_split(data, test_size=0.15)

In [18]:
algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [19]:
# Сформируем прогноз на основе полученной модели
test_pred = algo.test(testset)

In [20]:
# Проверим качество прогноза
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9618


0.9617706417228883

In [22]:
from surprise import SVD, SVDpp
from surprise.model_selection import cross_validate

In [23]:
algo = SVD()
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8743  0.8745  0.8747  0.8725  0.8730  0.8738  0.0009  
Fit time          9.66    9.91    10.28   9.89    9.85    9.92    0.20    
Test time         0.53    0.30    0.30    0.38    0.38    0.38    0.08    


{'test_rmse': array([0.87427118, 0.87451007, 0.87466373, 0.87249302, 0.8729856 ]),
 'fit_time': (9.655619621276855,
  9.913264274597168,
  10.275587558746338,
  9.892565965652466,
  9.8535635471344),
 'test_time': (0.5304009914398193,
  0.3000171184539795,
  0.3020172119140625,
  0.3840219974517822,
  0.38102197647094727)}

In [24]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
print(f'For SVD rmse : {accuracy.rmse(predictions)}')

RMSE: 0.8640
For SVD rmse : 0.8640439030065644


In [26]:
from surprise.model_selection import KFold

kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8786
RMSE: 0.8701
RMSE: 0.8778
RMSE: 0.8638
RMSE: 0.8754
